In [2]:
import pywren, json, requests, time, bs4
import pandas as pd
from re import findall
import numpy as np

In [2]:
data = pd.read_csv('complete_data.csv')
mobile = "https://mobile.twitter.com"
tar_accounts = list(set(data['twitter'])-{'-1', np.NAN, 'Twitter'})
print(len(tar_accounts))

691


In [3]:
def find_text_urls(tweets):
    temp = tweets.find('div', {'class': 'dir-ltr'})
    try: text = temp.text.strip()
    except AttributeError: return (None, None)
    urls = temp.find('a', {'class':'twitter_external_link'})
    if urls: urls = urls['data-url']
    return text, urls

def extract_texts(tweet_sp):
    t_urls = [find_text_urls(i) for i in tweet_sp.findAll('div', {'class': 'tweet-text'})]
    original_writers = tweet_sp.findAll('td', {'class': 'user-info'})[1:]
    original_writers = [i.find('a')['href'][:-4].strip('/') for i in original_writers]
    time = [i.text.strip() for i in tweet_sp.findAll('td', {'class': 'timestamp'})]
    return list(zip(original_writers, time, t_urls))

def scrape_user_tweets(username):
    base = f"{mobile}/{username}?"
    tar_url = f"{mobile}/{username}"
    results = []
    for r in range(300):
        response = requests.get(tar_url)
        for i in range(10):
            if response: break
            else: 
                time.sleep(10)
                response = requests.get(tar_url)
                
        if not response: break
        soup = bs4.BeautifulSoup(response.text, "html.parser")
        results.extend(extract_texts(soup))
        cursor = soup.find_all("div", "w-button-more")
        try: init = findall(r'href=(.*?)">', str(cursor))[0].split('?')[1]
        except IndexError: break
        tar_url = base + init
    return results

def scrape_user(username):
    url = f"{mobile}/{username}"
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.text, "html.parser")
    expert = dict()
    expert['bio'] = soup.find('div', {'class': 'dir-ltr'}).text.strip()
    expert["location"] = soup.find('div', {'class': 'location'}).text.strip()
    stat = soup.findAll('div', {'class': 'statnum'})
    expert['num_tweets'] = stat[0].text.replace(',', '')
    expert['num_following'] = stat[1].text.replace(',', '')
    expert['num_followers'] = stat[2].text.replace(',', '')
    expert['tweets'] = scrape_user_tweets(username)
    
    return {username: expert}

In [4]:
expert_data = json.load(open('expert_profile_tweets.json'))
len(set(tar_accounts) - expert_data.keys())

NameError: name 'tar_accounts' is not defined

In [7]:
extra = set(tar_accounts) - expert_data.keys()
pwex = pywren.default_executor(job_max_runtime = 500)
futures = pwex.map(scrape_user, extra)
print(len(futures))

14


In [9]:
dones, not_dones = pywren.wait(futures, pywren.ANY_COMPLETED)
print(len(dones), len(not_dones), end = '\r')
while len(not_dones) != 0:
    dones, not_dones = pywren.wait(futures, pywren.ANY_COMPLETED)
    print(len(dones), len(not_dones), end = '  \r')

In [10]:
from requests.exceptions import SSLError
#expert_data = dict()
for num, item in enumerate(dones):
    print(num, end = '\r')
    try: expert_data.update(item.result())
    except: continue
#json.dump(expert_data, open('expert_profile_tweets.json', 'w'))

## Write to File

In [8]:
expert_data = json.load(open('expert_profile_tweets.json'))
data['twitter_bio'] = data['twitter'].apply(lambda x: expert_data[x]['bio'] if x in expert_data else np.NAN)
data['location'] = data['twitter'].apply(lambda x: expert_data[x]['location'] if x in expert_data else np.NAN)
data['num_tweets'] = data['twitter'].apply(lambda x: expert_data[x]['num_tweets'] if x in expert_data else np.NAN)
data['num_following'] = data['twitter'].apply(lambda x: expert_data[x]['num_following'] if x in expert_data else np.NAN)
data['num_followers'] = data['twitter'].apply(lambda x: expert_data[x]['num_followers'] if x in expert_data else np.NAN)

In [13]:
data.to_csv('complete_data.csv', index=False)
max([len(expert_data[expert]['tweets']) for expert in expert_data])

3250

In [8]:
expert_data['BarryRabe']

{'bio': 'Environmental policy @fordschool @napawash @BrookingsGov   Last book: Can We Price Carbon?  Next up: Trump, the Administrative Presidency & Federalism (9/2020)',
 'location': 'University of Michigan',
 'num_tweets': '1202',
 'num_following': '98',
 'num_followers': '1119',
 'tweets': [['BarryRabe',
   'Sep 16',
   ['A Forum for Federalism That’s Sorely Missed governing.com/now/A-Forum-fo…',
    'https://www.governing.com/now/A-Forum-for-Federalism-Thats-Sorely-Missed.html']],
  ['BarryRabe',
   'Sep 11',
   ['Grateful for the opportunity to discuss a wide range of salient topics with @LarryHogan for @FordSchool Policy Talks twitter.com/fordschool/sta…',
    'https://twitter.com/fordschool/status/1304455333855793152']],
  ['MichaelRoss7',
   'Sep 8',
   ['I’ll be the series editor & joined by a stellar Editorial Board: @bernauereth Xun Cao @NavrozDubash @hochstet @KeohaneR @mmildenberger Helen Milner @mullinmeg @BarryRabe Kenneth Scheve @leahstokes @dustintingley 2/5',
    None

## Brief Analysis

In [9]:
from datetime import datetime
def check_2020(date):
    if len(date.split(' ')) > 2: return False
    try: assume_time = datetime.strptime(date+' 2020', '%b %d %Y')
    except ValueError: return False ## it is 2020 but after Aug 31
    if assume_time > datetime.strptime('Aug 31 2020', '%b %d %Y'): return False
    return True

In [64]:
from collections import Counter
ret_at_dict = dict()
for expert in expert_data:
    ret_at_dict[expert] = dict()
    ret, at = [], []
    for i in expert_data[expert]['tweets']:
        if not check_2020(i[1]): break
        if i[0] != expert: ret.append(i[0])
        at.extend([x[1:].strip() for x in i[2][0].split(' ') 
                   if len(x) > 1 and x[0]=='@'])
    ret_at_dict[expert]['retweet'] = dict(Counter(ret))
    ret_at_dict[expert]['at'] = dict(Counter(at))
    ret_at_dict[expert]['hrefs'] = set([i[2][1] for i in 
                                    expert_data[expert]['tweets']])

In [12]:
expert_data['_AllisonJordan']['tweets'][-1]

['CAPActionGVP',
 'Jun 15',
 ['The U.S Supreme Court opts to NOT take up 10 Second Amendment cases - which could have wiped out a suite life saving gun violence prevention measures and ushered in a wave of dangerous gun laws.\n\nMake no mistake, this is a HUGE win for public safety.\n\nreuters.com/article/us-usa…',
  'https://www.reuters.com/article/us-usa-court-guns/us-supreme-court-declines-to-hear-gun-rights-cases-idUSKBN23M1Y0']]

In [82]:
ret_at_dict['_AllisonJordan']['hrefs']

{'http://www.ncai.org/tribalnations/introduction/Indian_Country_101_Updated_February_2019.pdf',
 'https://twitter.com/SCOTUSblog/status/1273617165443387392',
 'https://twitter.com/ajplus/status/1269673903301033986',
 'https://twitter.com/indianz/status/1273268936574996483',
 'https://www.americanprogress.org/?p=486480',
 'https://www.americanprogress.org/issues/guns-crime/reports/2020/08/06/488686/gun-industry-america/',
 'https://www.reuters.com/article/us-usa-court-guns/us-supreme-court-declines-to-hear-gun-rights-cases-idUSKBN23M1Y0'}

In [14]:
count = 0
for expert in expert_data:
    try:last_date = expert_data[expert]['tweets'][-10][1]
    except IndexError: continue
    if check_2020(last_date): 
        count+=1
len(expert_data)-count

587

In [15]:
587/len(expert_data)

0.8657817109144543

In [69]:
def match(x, typ):
    try: return ret_at_dict[x[0]][typ][x[1]]
    except KeyError: return 0
def share_links(x):
    if x[0] not in ret_at_dict: a = set()
    else: a = set(ret_at_dict[x[0]]['hrefs'])
    if x[1] not in ret_at_dict: b = set()
    else: b = set(ret_at_dict[x[1]]['hrefs'])
    return len(a&b)

In [3]:
data = pd.read_csv('data4analysis.csv')

In [35]:
data['retweeted'] = data[['from', 'to']].apply(lambda x: match(x, 'retweet'), axis=1)
data['at'] = data[['from', 'to']].apply(lambda x: match(x, 'at'), axis=1)

In [70]:
data['share_links'] = data[['from', 'to']].apply(share_links, axis=1)

In [80]:
data.to_csv('data4analysis.csv', index=False)

In [72]:
Counter(data[data['share_links']!=0]['is_edge'])

Counter({False: 340948, True: 16116})

In [75]:
Counter(data[data['is_edge']]['share_links'])[0]

147

In [76]:
len(data[data['is_edge']])

16263

In [55]:
sum(data[data['retweeted']!=0]['is_edge'])/len(data[data['retweeted']!=0])

0.8028169014084507

In [79]:
data.columns

Index(['from', 'to', 'is_edge', 'expert_id_x', 'name_x', 'twitter_x',
       'institution_x', 'num_opinions_x', 'num_testimony_x', 'num_press_x',
       'num_external_x', 'is_resident_x', 'is_senior_x', 'is_leadership_x',
       'is_phd_x', 'is_law_x', 'num_tweets_x', 'num_following_x',
       'num_followers_x', 'expert_id_y', 'name_y', 'twitter_y',
       'institution_y', 'num_opinions_y', 'num_testimony_y', 'num_press_y',
       'num_external_y', 'is_resident_y', 'is_senior_y', 'is_leadership_y',
       'is_phd_y', 'is_law_y', 'num_tweets_y', 'num_following_y',
       'num_followers_y', 'share_institution', 'more_tweets', 'more_followers',
       'more_following', 'higher_rank', 'head', 'higher_degree',
       'topic_similarity', 'tweet_freq_x', 'tweet_freq_y', 'tweet_freq_diff',
       'tweet_similarity', 'retweeted', 'at', 'share_links'],
      dtype='object')

In [78]:
import statsmodels.api as sm
X = data.drop(['from', 'to', 'is_edge', ], axis=1)
y = data['share_links']

X_const = sm.add_constant(X)
logit = sm.Logit(y, X_const.astype(float)).fit()
print(logit.summary())

ValueError: could not convert string to float: 'samuel-j-abrams'

In [56]:
sum(data[data['at']!=0]['is_edge'])/len(data[data['at']!=0])

0.5945945945945946

In [57]:
data[(data['retweeted']!=0)][['institution_x', 'institution_y']]

,institution_x,institution_y
9273,aei,aei
19222,aei,aei
21015,aei,aei
29525,cap,aei
42027,aei,aei
...,...,...
362015,cap,cap
373257,cap,cap
373874,cap,cap
373875,cap,cap


In [60]:
cut = data[(data['at']!=0)]
sum(cut['institution_x']==cut['institution_y']), len(cut)

(67, 74)

In [61]:
67/74

0.9054054054054054